# Azure AI Agent with Hosted MCP Example

This notebook demonstrates integration of Azure AI Agents with hosted Model Context Protocol (MCP) servers, including user approval workflows for function call security.

## Features Covered:
- Hosted MCP tool integration
- User approval workflows for function calls
- Thread-based conversation management
- Azure AI observability setup
- Microsoft Learn MCP server integration

## Prerequisites

Before running this notebook, ensure you have:

1. **Azure AI Project**: Access to an Azure AI Foundry project with deployed models
2. **Authentication**: Azure CLI installed and authenticated (`az login --use-device-code`)
3. **Environment Variables**: Set up your `.env` file with connection details
4. **Dependencies**: Required agent-framework packages installed

If you need to use a different tenant, specify the tenant ID:
```bash
az login --tenant <tenant-id>
```

## Import Libraries

Import the required libraries for Azure AI agent functionality with MCP integration.

In [ ]:
import os
from pathlib import Path
import asyncio
from typing import Any

from agent_framework import AgentProtocol, AgentThread, HostedMCPTool, ChatMessage
from agent_framework.azure import AzureAIAgentClient
from azure.identity.aio import AzureCliCredential
from dotenv import load_dotenv  # For loading environment variables from .env file

# Get the path to the .env file which is in the parent directory
notebook_path = Path().absolute()  # Get absolute path of current notebook
parent_dir = notebook_path.parent  # Get parent directory
load_dotenv('../../.env')  # Load environment variables from .env file

In [ ]:
# This cell is for demonstration purposes only - actual async client setup is in the main functions
# The main functions use the proper async imports from the previous cell

print("📝 Note: The actual client initialization happens in the main functions using async context managers")
print("✅ Environment variables and async imports are ready for use")

## Check Environment Variables

Let's verify that the required environment variables are set:

In [ ]:
# Check required environment variables
required_vars = ["AZURE_AI_PROJECT_ENDPOINT", "AZURE_AI_MODEL_DEPLOYMENT_NAME"]
missing_vars = []

for var in required_vars:
    value = os.getenv(var)
    if value:
        print(f"✅ {var}: {value[:50]}..." if len(value) > 50 else f"✅ {var}: {value}")
    else:
        print(f"❌ {var}: Not set")
        missing_vars.append(var)

if missing_vars:
    print(f"\n⚠️  Please set the following environment variables: {', '.join(missing_vars)}")
else:
    print("\n✅ All required environment variables are set!")

## User Approval Handler

Define a function to handle user approvals for function calls, providing security for MCP tool executions:

In [ ]:
async def handle_approvals_with_thread(query: str, agent: "AgentProtocol", thread: "AgentThread"):
    """Handle user approvals for function calls in a thread context.
    
    This function manages the approval workflow for MCP tool function calls,
    providing security by requiring user consent before executing functions.
    """
    result = await agent.run(query, thread=thread, store=True)
    
    while len(result.user_input_requests) > 0:
        new_input: list[Any] = []
        
        for user_input_needed in result.user_input_requests:
            print(f"\n🔐 User Input Request for function from {agent.name}:")
            print(f"   Function: {user_input_needed.function_call.name}")
            print(f"   Arguments: {user_input_needed.function_call.arguments}")
            
            # In a notebook environment, we'll automatically approve for demonstration
            # In production, you would implement proper user input mechanisms
            user_approval = "y"  # Auto-approve for demo
            print(f"   📝 Auto-approving for demonstration: {user_approval}")
            
            new_input.append(
                ChatMessage(
                    role="user",
                    contents=[user_input_needed.create_response(user_approval.lower() == "y")],
                )
            )
        
        result = await agent.run(new_input, thread=thread, store=True)
    
    return result

## Main MCP Integration Example

This example demonstrates the complete workflow:
1. Set up Azure AI client with observability
2. Create an agent with hosted MCP tools
3. Execute queries with user approval workflow
4. Demonstrate thread-based conversation management

In [ ]:
async def main() -> None:
    """Example showing Hosted MCP tools for an Azure AI Agent."""
    async with (
        AzureCliCredential() as credential,
        AzureAIAgentClient(async_credential=credential) as chat_client,
    ):
        # Enable azure-ai observability
        print("🔍 Setting up Azure AI observability...")
        await chat_client.setup_azure_ai_observability()
        print("✅ Azure AI observability enabled")
        
        # Create agent with hosted MCP tool
        print("\n🤖 Creating agent with Microsoft Learn MCP server...")
        agent = chat_client.create_agent(
            name="DocsAgent",
            instructions="You are a helpful assistant that can help with Microsoft documentation questions.",
            tools=HostedMCPTool(
                name="Microsoft Learn MCP",
                url="https://learn.microsoft.com/api/mcp",
            ),
        )
        print(f"✅ Created agent: {agent.name}")
        
        # Create a new thread for conversation
        thread = agent.get_new_thread()
        print(f"📝 Created new conversation thread")
        
        # First query
        query1 = "How to create an Azure storage account using az cli?"
        print(f"\n=== Query 1 ===")
        print(f"🤔 User: {query1}")
        result1 = await handle_approvals_with_thread(query1, agent, thread)
        print(f"🤖 {agent.name}: {result1.text}")
        
        print("\n=======================================")
        
        # Second query
        query2 = "What is Microsoft Agent Framework?"
        print(f"\n=== Query 2 ===")
        print(f"🤔 User: {query2}")
        result2 = await handle_approvals_with_thread(query2, agent, thread)
        print(f"🤖 {agent.name}: {result2.text}")

## Execute the Example

Run the main function to see the hosted MCP integration in action:

In [ ]:
# Run the main function
await main()

## Key Takeaways

1. **Hosted MCP Tools**: Enable integration with external Model Context Protocol servers
2. **User Approval Workflows**: Provide security by requiring consent for function calls
3. **Thread Management**: Maintain conversation context across multiple queries
4. **Azure AI Observability**: Built-in monitoring and tracing for agent interactions
5. **Microsoft Learn Integration**: Access to comprehensive Microsoft documentation
6. **Error Handling**: Robust error handling for production scenarios

## Best Practices

1. **Security First**: Always implement proper approval workflows for function calls
2. **Observability**: Enable Azure AI observability for monitoring and debugging
3. **Thread Management**: Use threads to maintain conversation context
4. **Error Handling**: Implement comprehensive error handling for reliability
5. **Custom Approvals**: Tailor approval logic to your specific security requirements
6. **Resource Cleanup**: Properly manage agent and thread lifecycles

## Use Cases

- **Documentation Assistance**: AI-powered help with Microsoft technologies
- **Technical Support**: Automated support with human oversight
- **Knowledge Management**: Organizational knowledge base integration
- **Training and Education**: Interactive learning with documentation
- **Code Generation**: Context-aware code examples and templates
- **Compliance**: Secure function execution with approval workflows